In [13]:
!wget -N https://github.com/mishal9/Fraud-detection-in-graph-databases/blob/master/data/bs140513_032310.csv?raw=true


--2020-09-15 08:34:17--  https://github.com/mishal9/Fraud-detection-in-graph-databases/blob/master/data/bs140513_032310.csv?raw=true
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/mishal9/Fraud-detection-in-graph-databases/raw/master/data/bs140513_032310.csv [following]
--2020-09-15 08:34:18--  https://github.com/mishal9/Fraud-detection-in-graph-databases/raw/master/data/bs140513_032310.csv
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/mishal9/Fraud-detection-in-graph-databases/master/data/bs140513_032310.csv [following]
--2020-09-15 08:34:18--  https://raw.githubusercontent.com/mishal9/Fraud-detection-in-graph-databases/master/data/bs140513_032310.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 199.232.64.133
Connecting

In [14]:
ls

bs140513_032310.csv?raw=true  lost+found/  Untitled.ipynb


In [15]:
df = pd.read_csv('bs140513_032310.csv?raw=true')

In [16]:
df.head()

,step,customer,age,gender,zipcodeOri,merchant,zipMerchant,category,amount,fraud
0,0,'C1093826151','4','M','28007','M348934600','28007','es_transportation',4.55,0
1,0,'C352968107','2','M','28007','M348934600','28007','es_transportation',39.68,0
2,0,'C2054744914','4','F','28007','M1823072687','28007','es_transportation',26.89,0
3,0,'C1760612790','3','M','28007','M348934600','28007','es_transportation',17.25,0
4,0,'C757503768','5','M','28007','M348934600','28007','es_transportation',35.72,0


In [17]:
unique_print_str =  ""
for column in df:
    unique_print_str += " |{}: {}| ".format(column, df[column].unique().size)
    #print(df[column].unique().size)
print('---------- Number of unique values per feature ----------')
print(unique_print_str)

---------- Number of unique values per feature ----------
 |step: 180|  |customer: 4112|  |age: 8|  |gender: 4|  |zipcodeOri: 1|  |merchant: 50|  |zipMerchant: 1|  |category: 15|  |amount: 23767|  |fraud: 2| 


In [18]:
total = df.shape[0]
normal = df[df.fraud == 0].customer.count()
fraudulent = total - normal

print("The total number of datapoints are {}".format(total))
print("The number of non-fraudulent datapoints are {}, equal to {} % of the dataset".format(normal, round(100 *normal/total, 2)))
print("The number of fraudulent datapoints are {}, equal to {} % of the dataset".format(fraudulent, round(100 *fraudulent/total,2)))

The total number of datapoints are 594643
The number of non-fraudulent datapoints are 587443, equal to 98.79 % of the dataset
The number of fraudulent datapoints are 7200, equal to 1.21 % of the dataset


In [19]:
# split the features and labels
label = df.fraud
features = df.drop('fraud', axis = 1)
features.head()

,step,customer,age,gender,zipcodeOri,merchant,zipMerchant,category,amount
0,0,'C1093826151','4','M','28007','M348934600','28007','es_transportation',4.55
1,0,'C352968107','2','M','28007','M348934600','28007','es_transportation',39.68
2,0,'C2054744914','4','F','28007','M1823072687','28007','es_transportation',26.89
3,0,'C1760612790','3','M','28007','M348934600','28007','es_transportation',17.25
4,0,'C757503768','5','M','28007','M348934600','28007','es_transportation',35.72


In [20]:
# check if empty values...
df.isnull().values.any()

False

In [21]:
features =  features.drop(['zipcodeOri', 'zipMerchant', 'customer'], axis = 1)
features.head()

,step,age,gender,merchant,category,amount
0,0,'4','M','M348934600','es_transportation',4.55
1,0,'2','M','M348934600','es_transportation',39.68
2,0,'4','F','M1823072687','es_transportation',26.89
3,0,'3','M','M348934600','es_transportation',17.25
4,0,'5','M','M348934600','es_transportation',35.72


In [22]:
from sklearn.preprocessing import MinMaxScaler

# Initialize a scaler, then apply it to the features
scaler = MinMaxScaler()

df[['amount', 'fraud']] = scaler.fit_transform(df[['amount', 'fraud']])
features.amount = df.amount


# Show an example of a record with scaling applied
features.head()

,step,age,gender,merchant,category,amount
0,0,'4','M','M348934600','es_transportation',0.000546
1,0,'2','M','M348934600','es_transportation',0.004764
2,0,'4','F','M1823072687','es_transportation',0.003228
3,0,'3','M','M348934600','es_transportation',0.002071
4,0,'5','M','M348934600','es_transportation',0.004288


In [23]:
features_final = pd.get_dummies(features)

In [24]:
features_final.head()

,step,amount,age_'0',age_'1',age_'2',age_'3',age_'4',age_'5',age_'6',age_'U',...,category_'es_home',category_'es_hotelservices',category_'es_hyper',category_'es_leisure',category_'es_otherservices',category_'es_sportsandtoys',category_'es_tech',category_'es_transportation',category_'es_travel',category_'es_wellnessandbeauty'
0,0,0.000546,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,0,0.004764,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,0,0.003228,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,0,0.002071,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,0,0.004288,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0


In [25]:
features_final.shape

(594643, 79)

In [26]:
from sklearn.model_selection import * 
from sklearn.metrics import fbeta_score, accuracy_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

In [27]:
kf = KFold(n_splits=2)
fold_betas = []
fold_accuracy = []
kf.get_n_splits(features_final)

for train_index, test_index in kf.split(features_final):
    X_train, X_test = features_final.iloc[train_index], features_final.iloc[test_index]
    y_train, y_test = label.iloc[train_index], label.iloc[test_index]
    
    clfSVM = SVC()
    clfSVM.fit(X_train,y_train)
    clfRandomForest = RandomForestClassifier()
    clfRandomForest.fit(X_train,y_train)
    
    predictionsSVM = clfSVM.predict(X_test)
    predictionsRF = clfRandomForest.predict(X_test)
    
    fold_betas.append({"SVM": fbeta_score( y_test, predictionsSVM, average='macro', beta=1),
                       "RF": fbeta_score( y_test, predictionsRF, average='macro', beta=1) })
    fold_accuracy.append({"SVM": accuracy_score( y_test, predictionsSVM),
                       "RF": accuracy_score(y_test, predictionsRF) })

In [28]:
def calculate_average_for_fold(fold_data, model_names):
    model_1_sum = 0
    model_2_sum = 0

    for dat in fold_data:
        model_1_sum += dat[model_names[0]]/len(fold_data)
        model_2_sum += dat[model_names[1]]/len(fold_data)

    print("Model {} average: {}".format(model_names[0], model_1_sum))
    print("Model {} average: {}".format(model_names[1], model_2_sum))

In [32]:
#from sklearn.grid_search import GridSearchCV

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.model_selection import train_test_split

# Split the 'features' and 'income' data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features_final, 
                                                    label, 
                                                    test_size = 0.2, 
                                                    random_state = 0)

# TODO: Initialize the classifier
clf = RandomForestClassifier()
clf2 = RandomForestClassifier()

clf2.fit(X_train, y_train)

predictions = clf2.predict(X_test)

# TODO: Create the parameters list you wish to tune, using a dictionary if needed.
# HINT: parameters = {'parameter_1': [value1, value2], 'parameter_2': [value1, value2]}
parameters = {'n_estimators': [5, 10 , 100],
              'min_samples_split': [2, 10, 50],
              'max_features': ["sqrt", "log2"],
             }

# TODO: Make an fbeta_score scoring object using make_scorer()
scorer = make_scorer(fbeta_score, beta=1)

# TODO: Perform grid search on the classifier using 'scorer' as the scoring method using GridSearchCV()
grid_obj = GridSearchCV(clf, param_grid=parameters, scoring=scorer)

# TODO: Fit the grid search object to the training data and find the optimal parameters using fit()
grid_fit = grid_obj.fit(X_train, y_train)

# Get the estimator
best_clf = grid_fit.best_estimator_

# Make predictions using the unoptimized and model
predictions = (clf.fit(X_train, y_train)).predict(X_test)
best_predictions = best_clf.predict(X_test)

# Report the before-and-afterscores
print("Unoptimized model\n------")
print("Accuracy score on testing data: {:.4f}".format(accuracy_score(y_test, predictions)))
print("F-score on testing data: {:.4f}".format(fbeta_score(y_test, predictions, beta = 1)))
print("\nOptimized Model\n------")
print("Final accuracy score on the testing data: {:.4f}".format(accuracy_score(y_test, best_predictions)))
print("Final F-score on the testing data: {:.4f}".format(fbeta_score(y_test, best_predictions, beta = 1)))

Unoptimized model
------
Accuracy score on testing data: 0.9957
F-score on testing data: 0.8089

Optimized Model
------
Final accuracy score on the testing data: 0.9959
Final F-score on the testing data: 0.8182


In [33]:
best_clf

RandomForestClassifier(max_features='sqrt', min_samples_split=50)